# Data pre-processing

In [7]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
#------------------------------#
# loaddta(): load data
# standard(): normalise data
#------------------------------#


def loaddata():
    dataset_mnist = np.load('mnist_dataset.npz')
    dataset_cifar = np.load('cifar_dataset.npz')

    Xtr_mnist = dataset_mnist['Xtr']
    Str_mnist = dataset_mnist['Str']
    Xts_mnist = dataset_mnist['Xts']
    Yts_mnist = dataset_mnist['Yts']

    Xtr_cifar = dataset_cifar['Xtr']
    Str_cifar = dataset_cifar['Str']
    Xts_cifar = dataset_cifar['Xts']
    Yts_cifar = dataset_cifar['Yts']

    return Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar


def standard(x_train, x_test):
    '''
    Note:
         Use the std and mean from the training data and implement
         it to the validation data and test data
    RETRUN:
        Flattened and standard(Z-score normalization) training data
        The features size is 3072 or 784
    '''
    std = np.std(x_train, keepdims=True)
    mean = np.mean(x_train, keepdims=True)
    x_train = (x_train-mean)/std
    x_test = (x_test-mean)/std
    return x_train, x_test

#============#
# split_data
#============#

def split_data(x_train, y_train, part=0.8):
    '''
    Shuffle then data and then split the data accroding to the part parameters
    '''
    all_index = np.arange(y_train.shape[0])
    np.random.shuffle(all_index)
    
    train_data_size = int(y_train.shape[0] * part)
    
    train_index = all_index[0:train_data_size]
    val_index= all_index[train_data_size:]
    
    
    x_vali = x_train[val_index]
    y_vali = y_train[val_index]

    x_train = x_train[train_index]
    y_train = y_train[train_index]

    return x_train, y_train, x_vali, y_vali,train_index

#==================#
# One-hot embedding
#==================#

def one_hot_coding(y_train, y_test):
    '''
    Encode label to one-hot catogory
    '''
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)
    return y_train, y_test

def compute_acc(y_ture, y_pred):
    acc = np.sum(y_ture == np.argmax(y_pred, axis=1))/len(y_ture)
    return round(acc, 3)


# CNN model

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import tensorflow as tf
import numpy as np


def train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                    lr=0.0001, decay=0.000001, batchsize=256, epochs=1):
    '''
    x_train: original data
    y_train: one-hot encoding

    Traning resnet model on label nosie data
    Usesd for estimate the T
    '''
    # data is mnist
    if x_train.shape[3] == 1:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(28,28,1)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),

            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])
        
    # data is cifar
    else:
        model = Sequential([
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu',input_shape=(32,32,3)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (2, 2), use_bias=True, padding='same', activation='relu'),
            Flatten(),
            Dense(1000, activation='relu', use_bias=True),
            Dense(500, activation='relu', use_bias=True),
            Dense(2, activation=tf.nn.softmax)
        ])

    adm = optimizers.Adam(lr=lr, decay=decay)
    if TYPE == 'with_noise':
        model.compile(loss='categorical_crossentropy',
                      optimizer=adm,
                      metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batchsize, \
                        epochs=epochs)
    elif TYPE == 'without_noise':
        model.compile(loss='categorical_crossentropy',
                      optimizer=adm,
                      metrics=['accuracy'])
        
        history = model.fit(x_train, y_train, batch_size=batchsize,sample_weight = weights.flatten(), \
                        epochs=epochs)
    return model, history


rho0 = 0.2
rho1 = 0.4
def estimateBeta(S,prob,rho0,rho1):
    n = len(S)
    beta = np.zeros((n,1))
    for i in range(n):
        if S[i]==1:
            beta[i] = (prob[i][1]-rho0)/((1-rho0-rho1)*prob[i][1]+1e-5)
        else:
            beta[i] = (prob[i][0]-rho1)/((1-rho0-rho1)*(prob[i][0])+1e-5)

    return beta


# Benchmark function 

## Benchmark of CNN without loss function correction

In [5]:
Xtr_mnist, Str_mnist, Xts_mnist, Yts_mnist, Xtr_cifar, Str_cifar, Xts_cifar, Yts_cifar = loaddata()
Xtr_mnist,Xts_mnist = standard(Xtr_mnist,Xts_mnist)
Xtr_mnist = Xtr_mnist.reshape(Xtr_mnist.shape[0], 28, 28,1)
Xts_mnist = Xts_mnist.reshape(Xts_mnist.shape[0], 28, 28,1)
y_train_o, y_test = one_hot_coding(Str_mnist,Yts_mnist)

print('x_train {}, y_train {} \nXts_mnist {}, y_test {}'.format(
    Xtr_mnist.shape,y_train_o.shape,Xts_mnist.shape,y_test.shape))




FileNotFoundError: [Errno 2] No such file or directory: 'mnist_dataset.npz'

In [4]:
for_flip_rate_1_l = []
for_flip_rate_0_l = []
acc_l = []
run_times = 10
x_train_x_results = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_mnist,y_train_o,part=0.8)
    model_noise,history_noise = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                        lr=0.00001, decay=0.000001, batchsize=1024, epochs=10)
    
    train_x_result = model_noise.predict(Xtr_mnist)

    for_flip_rate_1_l.append(np.min(train_x_result[:,0]))
    for_flip_rate_0_l.append(np.min(train_x_result[:,1]))

    l,a=model_noise.evaluate(Xts_mnist,y_test)
    
    acc_l.append(a)
    x_train_x_results.append(train_x_result)

Epoch 1/10
8000/8000 [==============================] - 2s 209us/step - loss: 0.6669 - acc: 0.5981
Epoch 2/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6486 - acc: 0.6206
Epoch 3/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6369 - acc: 0.6300
Epoch 4/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6276 - acc: 0.6305
Epoch 5/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6199 - acc: 0.6423
Epoch 6/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6145 - acc: 0.6505
Epoch 7/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6098 - acc: 0.6621
Epoch 8/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6070 - acc: 0.6636
Epoch 9/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6044 - acc: 0.6690
Epoch 10/10
2000/2000 [==============================] - 0s 68us/step
Epoch 1/10
8000/8000 [=========================

In [5]:
print(for_flip_rate_0_l)
print(for_flip_rate_1_l)
print(acc_l)
print(sum(acc_l)/len(acc_l))
print(np.array(acc_l).std())
# print(x_train_x_results)

[0.118475705, 0.112255834, 0.10182339, 0.12633117, 0.118924625, 0.12552512, 0.1293975, 0.11892501, 0.11038665, 0.11261818]
[0.22113787, 0.22626345, 0.2290852, 0.18226421, 0.2148817, 0.21624699, 0.23876645, 0.22864051, 0.21219683, 0.2024535]
[0.868, 0.859, 0.8375, 0.875, 0.873, 0.8815, 0.799, 0.8755, 0.8615, 0.875]
0.8605
0.023692825918408277


In [6]:
for i in range(len(x_train_x_results)):
    if i == 0:
        total_x_result = np.copy(x_train_x_results[i])
    else:
        total_x_result += x_train_x_results[i]
        
train_x_result = total_x_result/(run_times)

all_weights = estimateBeta(Str_mnist, train_x_result, rho0, rho1)
for i in range(len(all_weights)):
    if all_weights[i] < 0:
        all_weights[i] = 0.0

In [7]:
acc_l_without = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_mnist,y_train_o,part=0.8)
    
    weights = all_weights[split_index]
    
    model, history = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='without_noise',
                        lr=0.00001, decay=0.000001, batchsize=1024, epochs=10)
    
#     model.evaluate(Xts_mnist,y_test)
    l,a=model.evaluate(Xts_mnist,y_test)
    acc_l_without.append(a)


Epoch 1/10
8000/8000 [==============================] - 1s 125us/step - loss: 0.6636 - acc: 0.5694
Epoch 2/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.6046 - acc: 0.6053
Epoch 3/10
8000/8000 [==============================] - 0s 33us/step - loss: 0.5550 - acc: 0.6440
Epoch 4/10
8000/8000 [==============================] - 0s 36us/step - loss: 0.5105 - acc: 0.6555
Epoch 5/10
8000/8000 [==============================] - 0s 33us/step - loss: 0.4707 - acc: 0.6618
Epoch 6/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4353 - acc: 0.6671
Epoch 7/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.4059 - acc: 0.6707
Epoch 8/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3819 - acc: 0.6678
Epoch 9/10
8000/8000 [==============================] - 0s 32us/step - loss: 0.3626 - acc: 0.6693
Epoch 10/10
2000/2000 [==============================] - 0s 124us/step
Epoch 1/10
8000/8000 [========================

In [8]:
print(acc_l_without)
print(sum(acc_l_without)/len(acc_l_without))
print(np.array(acc_l_without).std())

[0.9245, 0.9235, 0.925, 0.93, 0.9215, 0.921, 0.9255, 0.9305, 0.925, 0.9235]
0.9250000000000002
0.002974894956128708


In [9]:


Xtr_cifar,Xts_cifar = standard(Xtr_cifar,Xts_cifar)

Xtr_cifar = Xtr_cifar.reshape(Xtr_cifar.shape[0], 32,32,3)
Xts_cifar = Xts_cifar.reshape(Xts_cifar.shape[0], 32,32,3)

y_train_o, y_test = one_hot_coding(Str_cifar,Yts_cifar)

print('x_train {}, y_train {} \nXts_cifar {}, y_test {}'.format(
    Xtr_cifar.shape,y_train_o.shape,Xts_cifar.shape,y_test.shape))




x_train (10000, 32, 32, 3), y_train (10000, 2) 
Xts_cifar (2000, 32, 32, 3), y_test (2000, 2)


In [10]:
for_flip_rate_1_l = []
for_flip_rate_0_l = []
acc_l = []
run_times = 5
x_train_x_results = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_cifar,y_train_o,part=0.8)
    model_noise,history_noise = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='with_noise',
                        lr=0.00005, decay=0.000001, batchsize=1024, epochs=10)
    
    train_x_result = model_noise.predict(Xtr_cifar)

    for_flip_rate_1_l.append(np.min(train_x_result[:,0]))
    for_flip_rate_0_l.append(np.min(train_x_result[:,1]))

    l,a=model_noise.evaluate(Xts_cifar,y_test)
    
    acc_l.append(a)
    x_train_x_results.append(train_x_result)

Epoch 1/10
8000/8000 [==============================] - 2s 209us/step - loss: 0.6597 - acc: 0.6006
Epoch 2/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.6299 - acc: 0.6465
Epoch 3/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.6178 - acc: 0.6591
Epoch 4/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.6067 - acc: 0.6760
Epoch 5/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.5975 - acc: 0.6919
Epoch 6/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.5890 - acc: 0.6965
Epoch 7/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.5785 - acc: 0.7090
Epoch 8/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.5714 - acc: 0.7194
Epoch 9/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.5586 - acc: 0.7276
Epoch 10/10
2000/2000 [==============================] - 0s 220us/step
Epoch 1/10
8000/8000 [========================

In [11]:
print(for_flip_rate_0_l)
print(for_flip_rate_1_l)
print(acc_l)


print(sum(acc_l)/len(acc_l))
print(np.array(acc_l).std())
# print(x_train_x_results)

[0.029632399, 0.027437404, 0.03711124, 0.04075333, 0.028904041]
[0.13930126, 0.12055889, 0.15325263, 0.14035507, 0.14672887]
[0.836, 0.7735, 0.7765, 0.845, 0.795]
0.8051999999999999
0.029884109489827528


In [12]:
for i in range(len(x_train_x_results)):
    if i == 0:
        total_x_result = np.copy(x_train_x_results[i])
    else:
        total_x_result += x_train_x_results[i]
        
train_x_result = total_x_result/(run_times)

all_weights = estimateBeta(Str_cifar, train_x_result, rho0, rho1)
for i in range(len(weights)):
    if all_weights[i] < 0:
        all_weights[i] = 0.0

In [13]:
acc_l_without = []
for i in range(run_times):
    x_train, y_train, x_vali, y_vali,split_index = split_data(Xtr_cifar,y_train_o,part=0.8)
    
    weights = all_weights[split_index]
    
    model, history = train_cnn_model(x_train, y_train, x_vali, y_vali, TYPE='without_noise',
                        lr=0.00005, decay=0.000001, batchsize=1024, epochs=10)

    l,a=model.evaluate(Xts_cifar,y_test)
    acc_l_without.append(a)


Epoch 1/10
8000/8000 [==============================] - 2s 216us/step - loss: 0.5679 - acc: 0.5880
Epoch 2/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.4787 - acc: 0.6269
Epoch 3/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.4476 - acc: 0.6441
Epoch 4/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.4374 - acc: 0.6392
Epoch 5/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.4177 - acc: 0.6511
Epoch 6/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.4004 - acc: 0.6594
Epoch 7/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.3865 - acc: 0.6643
Epoch 8/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.3747 - acc: 0.6656
Epoch 9/10
8000/8000 [==============================] - 0s 45us/step - loss: 0.3640 - acc: 0.6704
Epoch 10/10
2000/2000 [==============================] - 1s 278us/step
Epoch 1/10
8000/8000 [========================

In [14]:
print(acc_l_without)
print(sum(acc_l_without)/len(acc_l_without))
print(np.array(acc_l_without).std())

[0.8805, 0.8865, 0.874, 0.868, 0.883]
0.8783999999999998
0.00661362230551457


In [8]:
# x_train, y_train, x_vali, y_vali = split_data1(Xtr_mnist,y_train_o,part=0.8)